In [10]:
%%capture
!pip install netCDF4 h5netcdf scipy


In [16]:
!file MERRA2_400.inst3_3d_asm_Np.20250101.nc4

MERRA2_400.inst3_3d_asm_Np.20250101.nc4: OpenPGP Secret Key


In [17]:
!gpg --list-packets MERRA2_400.inst3_3d_asm_Np.20250101.nc4


gpg: directory '/root/.gnupg' created
gpg: keybox '/root/.gnupg/pubring.kbx' created
gpg: packet(5) with unknown version 46
# off=0 ctb=96 tag=5 hlen=5 plen=2349107193
:key packet: [unknown version]


In [18]:
!gpg --decrypt MERRA2_400.inst3_3d_asm_Np.20250101.nc4 > MERRA2_400_decrypted.nc4


gpg: packet(5) with unknown version 46


In [19]:
import xarray as xr

file_path = "MERRA2_400_decrypted.nc4"

try:
    ds = xr.open_dataset(file_path, engine="netcdf4")  # Try netcdf4 first
    print("Successfully opened with netcdf4")
except ValueError:
    try:
        ds = xr.open_dataset(file_path, engine="h5netcdf")  # Try h5netcdf
        print("Successfully opened with h5netcdf")
    except ValueError:
        try:
            ds = xr.open_dataset(file_path, engine="scipy")  # Try scipy
            print("Successfully opened with scipy")
        except Exception as e:
            print("Failed to open the file with any backend:", e)

# If successfully loaded, print the dataset structure
if 'ds' in locals():
    print(ds)


OSError: [Errno -51] NetCDF: Unknown file format: '/content/MERRA2_400_decrypted.nc4'

In [7]:
with open("S3URLs.txt", "r") as f:
    s3_urls = [line.strip() for line in f.readlines()]

# Initialize S3 filesystem
fs = s3fs.S3FileSystem(anon=True)  # Set anon=False if authentication is required

# Open first file as an example
s3_url = s3_urls[0].replace("s3://", "")  # Remove the "s3://" prefix

try:
    # Open the file using s3fs
    with fs.open(s3_url, "rb") as f:
        dataset = Dataset(f, "r", memory=True)  # Load into memory

        print("File opened successfully from S3.")

        # Print available variables
        print("Variables:", dataset.variables.keys())

        # Read a specific variable (replace 'your_variable_name' with an actual one)
        var_name = list(dataset.variables.keys())[0]  # Get the first variable as an example
        data = dataset.variables[var_name][:]
        print(f"Data from {var_name}:", data)

        dataset.close()
except FileNotFoundError:
    print(f"Error: File not found at {s3_url}")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: Attempt to open non key-like path: 
